<a href="https://colab.research.google.com/github/SofiaRguez1020/Topicos_Industria_I/blob/main/Proyecto_SofiaRodriguez__AlejandraTrujillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Final: *MiBici*

**Nombre:** Sofia Daniela Rodriguez Saenz

**e-mail:** sofia.rodriguez5540@alumnos.udg.mx

**Nombre:** Alejandra Elizabeth Trujillo Navarro

**e-mail:** alejandra.trujillo2826@alumnos.udg.mx


# Descripción del proyecto
En este proyecto nos centraremos en el análisis y la visualización de datos
provenientes del sistema de bicicletas compartidas, el cual es una programa
ecológica y sostenible. A través de la recolección y el estudio de datos detallados sobre el uso de las bicicletas, estaciones, y otros factores relevantes como edad y género, este proyecto busca ofrecer detalles relevantes para mejorar el servicio de MiBici.
Al analizar cómo, cuándo y dónde se utilizan las bicicletas compartidas, podemos obtener detalles sobre las necesidades y preferencias de los usuarios, lo que a su vez puede guiar mejoras en la infraestructura, la distribución de las estaciones y la promoción de hábitos de transporte más sostenibles.
## Capa 1: Panel de control para generación del dataset
El propósito de esta capa es permitir al usuario generar o filtrar el conjunto de datos que será analizado. Se utilizarán los datos abiertos de MiBici, disponibles en la siguiente liga: https://www.mibici.net/es/datos-abiertos/. Será posible seleccionar rangos de fechas, estaciones específicas de bicicletas tanto de retiro como de
arribo, y segmentar los datos por género y edad de los usuarios.
## Capa 2: Representación gráfica del dataset
En esta capa, se presentará una representación gráfica del conjunto de datos
filtrado o generado en la Capa 1. Exhibiremos un mapa detallado de la ciudad,
señalando la ubicación de las estaciones y su respectivo número identificativo.
Además, incorporaremos una codificación por colores para destacar las estaciones de mayor uso.
## Capa 3: Panel de Control para Selección de Métricas y
Visualización de Resultados
Esta capa permitiría al usuario seleccionar diferentes métricas para analizar el dataset y visualizar los resultados de manera gráfica.
* Distancia total recorrida: Un histograma o gráfico de líneas mostrando la
distancia total recorrida por mes.
* Duración promedio de los viajes: Gráficos de barras que muestren la duración promedio de los viajes por estación.
* Densidad de uso por hora del día: Gráficos de líneas que muestren cómo
varía el uso de las bicicletas a lo largo del día, identificando las horas pico.

# MODULES

In [ ]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

import panel as pn
import panel.widgets as pnw

pn.extension('plotly')

# CLASSES

# FUNCTIONS

In [ ]:
def getDF(path):


#Obtener datos de MiBici

In [8]:
import requests
import os

#Creamos una carpeta para guardar los archivos
import errno
try:
    os.mkdir('MiBici')
except OSError as e:
    if e.errno != errno.EEXIST:
        raise
# URL donde se encuentran guardados los archivos de Ecobici
urls = {'Enero':r'https://www.mibici.net/site/assets/files/43035/datos_abiertos_2023_03.csv',
        'Febrero': r'https://www.mibici.net/site/assets/files/40197/datos_abiertos_2023_02.csv',
        'Marzo': r'https://www.mibici.net/site/assets/files/43035/datos_abiertos_2023_03.csv',
        'Abril': r'https://www.mibici.net/site/assets/files/46758/datos_abiertos_2023_04.csv',
        'Mayo': r'https://www.mibici.net/site/assets/files/46759/datos_abiertos_2023_05.csv',
        'Junio': r'https://www.mibici.net/site/assets/files/48116/datos_abiertos_2023_06.csv',
        'Julio': r'https://www.mibici.net/site/assets/files/49448/datos_abiertos_2023_07.csv',
        'Agosto': r'https://www.mibici.net/site/assets/files/51295/datos_abiertos_2023_08.csv',
        'Septiembre': r'https://www.mibici.net/site/assets/files/53344/datos_abiertos_2023_09.csv',
        'Octubre': r'https://www.mibici.net/site/assets/files/54935/datos_abiertos_2023_10.csv',
        'Noviembre': r'https://www.mibici.net/site/assets/files/57344/datos_abiertos_2023_11.csv',
        'Diciembre': r'https://www.mibici.net/site/assets/files/58715/datos_abiertos_2023_12.csv'
        }

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Realizar la solicitud GET para obtener archivo
#Descargaremos cada uno de los archivos de 2023 ya que 2024 no esta completo aun
for mes, url in urls.items():
    url_tmp = url
    print("Descargando datos: ", url, " ....")
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        # Abrir el archivo en modo binario y escribir los datos descargados
        ruta = 'MiBici/'+mes+'.csv'
        with open(ruta, "wb") as archivo:
            archivo.write(response.content)
        print(f"Archivo guardado como {mes}.csv")
    else:
        print(f"Error al descargar el archivo. Código de respuesta: {response.status_code}")


Descargando datos:  https://www.mibici.net/site/assets/files/43035/datos_abiertos_2023_03.csv  ....
Archivo guardado como Enero.csv
Descargando datos:  https://www.mibici.net/site/assets/files/40197/datos_abiertos_2023_02.csv  ....
Archivo guardado como Febrero.csv
Descargando datos:  https://www.mibici.net/site/assets/files/43035/datos_abiertos_2023_03.csv  ....
Archivo guardado como Marzo.csv
Descargando datos:  https://www.mibici.net/site/assets/files/46758/datos_abiertos_2023_04.csv  ....
Archivo guardado como Abril.csv
Descargando datos:  https://www.mibici.net/site/assets/files/46759/datos_abiertos_2023_05.csv  ....
Archivo guardado como Mayo.csv
Descargando datos:  https://www.mibici.net/site/assets/files/48116/datos_abiertos_2023_06.csv  ....
Archivo guardado como Junio.csv
Descargando datos:  https://www.mibici.net/site/assets/files/49448/datos_abiertos_2023_07.csv  ....
Archivo guardado como Julio.csv
Descargando datos:  https://www.mibici.net/site/assets/files/51295/datos_ab

In [ ]:
#Procederemos a guardar en 1 Dataframe y conmbinarlos en 1 solo archivo


In [ ]:
pn.extension()
#Define DF to track trajectory DF for both trajectoty plot and metric plot
global traj_global_df
traj_global_df = None

#Define widget layout
trajectory_type = pn.widgets.ToggleGroup(name='Panel', options=['BM', 'CRW', 'LF'], behavior="radio", width = 320, align='center')
n_steps = pn.widgets.IntSlider(name='Number of steps:', start=10, end=1000, value=500, width = 320, align = 'center')
x_pos = pn.widgets.IntInput(name='xInitPos', value=0, width = 150)
y_pos = pn.widgets.IntInput(name='yInitPos', value=0, width = 150)
speed = pn.widgets.IntSlider(name='Speed', start=1, end=15, value=5, width = 320, align='center')
metric = pn.widgets.Select(name='Metric', options=['PL','MSD'], width=320,align='center')
crw_exponent = pn.widgets.FloatInput(name='Cauchy coefficient', width=320, visible=False, align='center', step=1e-1, start=1e-1, end=1, value = 0.5)
alpha = pn.widgets.FloatInput(name='Alpha', visible=False, width=320, align='center', step=1e-1, start=1e-1, end=2, value = 1.5)

#Dinamically show or hide option for cauchy coef/levy exp
@pn.depends(trajectory_type, watch=True)
def show_input(trajectory_type):
    #Dinamically show or hide CRW and Alpha inputs
    if trajectory_type == 'BM':
        crw_exponent.visible = False
        alpha.visible = False
    elif trajectory_type == 'CRW':
        crw_exponent.visible = True
        alpha.visible = False
    elif trajectory_type == 'LF':
        crw_exponent.visible = True
        alpha.visible = True

#Define function to draw trajectory

@pn.depends(n_steps, x_pos, y_pos, speed, trajectory_type, crw_exponent, alpha)
def plot_traj(n_steps, x_pos, y_pos, speed,trajectory_type, crw_exponent, alpha):

  #Call trajectory function according to selection
  if trajectory_type == 'BM':
    traj_df = bm_2d(n_steps, speed, [x_pos, y_pos])
  elif trajectory_type == 'CRW':
    traj_df = crw(n_steps, speed, [x_pos, y_pos], crw_exponent)
  elif trajectory_type == 'LF':
    traj_df = levy_flight(n_steps, speed, [x_pos, y_pos], crw_exponent, alpha)

  #Save trajectory to aux var
  global traj_global_df
  traj_global_df = traj_df

  #Graph trajectory
  fig_traj_rw = go.Figure()
  fig_traj_rw.add_trace(go.Scatter3d(
                        x = traj_df.x_pos,
                        y = traj_df.y_pos,
                        z = traj_df.index,
                        name = trajectory_type,
                        mode = 'lines',
                        showlegend = True
                      ))

  #Add axis names and dinamically changes plot title
  fig_traj_rw.update_layout(
                    title = trajectory_type + ' Trajectory in 3D',
                    scene=dict(
                    xaxis_title='x_pos',
                    yaxis_title='y_pos',
                    zaxis_title='Time'))
  return fig_traj_rw


#Define function to draw metric plot

@pn.depends(n_steps, x_pos, y_pos, speed, trajectory_type, crw_exponent, alpha, metric)
def plot_metric(n_steps, x_pos, y_pos, speed, trajectory_type, crw_exponent, alpha, metric):
  #Get trajectory
  global traj_global_df
  if traj_global_df is None:
        return None
  if metric == 'MSD':
    metric_df = msd(traj_global_df, int(n_steps/2))
  elif metric == 'PL':
    metric_df = path_length(traj_global_df)
  #Graph trajectory
  fig_metric_rw = go.Figure()
  fig_metric_rw.add_trace(go.Scatter(
                        x = metric_df.index,
                        y = metric_df[metric],
                        name = trajectory_type,
                        mode = 'lines',
                        showlegend = True
                      ))

  #Add axis names and dinamically changes plot title
  fig_metric_rw.update_layout(
                    title = metric,
                    scene=dict(
                    xaxis_title='x_pos',
                    yaxis_title='y_pos',
                    zaxis_title='Time'))
  return fig_metric_rw


In [ ]:
dashboard = pn.Row(
    pn.WidgetBox(pn.pane.Markdown("# Panel options", align='center'),
                 trajectory_type,
                 n_steps,
                 pn.Row(x_pos, y_pos, align = 'center'),
                 speed,
                 metric,
                 crw_exponent,
                 alpha,
                 width= 380,
                 height = 500, align = 'center'),
    pn.Column(plot_traj, width = 500,height = 500, align='center'),
    pn.Column(plot_metric, width = 500,height = 500, align='center'),
    margin=100
)


In [ ]:
#Show dashboard embedded in colan
dashboard.servable()


Row(margin=100)
    [0] WidgetBox(align='center', height=500, sizing_mode='fixed', width=380)
        [0] Markdown(str, align='center')
        [1] RadioButtonGroup(align='center', name='Panel', options=['BM', 'CRW', 'LF'], value='BM', width=320)
        [2] IntSlider(align='center', end=1000, name='Number of steps:', start=10, value=500, width=320)
        [3] Row(align='center')
            [0] IntInput(name='xInitPos', width=150)
            [1] IntInput(name='yInitPos', width=150)
        [4] IntSlider(align='center', end=15, name='Speed', start=1, value=5, width=320)
        [5] Select(align='center', name='Metric', options=['PL', 'MSD'], value='MSD', width=320)
        [6] FloatInput(align='center', end=1, name='Cauchy coefficient', start=0.1, value=0.5, visible=False, width=320)
        [7] FloatInput(align='center', end=2, name='Alpha', start=0.1, value=1.2999999999999998, visible=False, width=320)
    [1] Column(align='center', height=500, sizing_mode='fixed', width=500)
        [0] ParamFunction(function, _pane=Plotly, defer_load=False)
    [2] Column(align='center', height=500, sizing_mode='fixed', width=500)
        [0] ParamFunction(function, _pane=Plotly, defer_load=False)

In [ ]:
#Show dashboard in localhost
dashboard.show()

Launching server at http://localhost:35439


## Notas

Link a codigo python unicamente se encuentra aqui:
https://github.com/SofiaRguez1020/Topicos_Industria_I/blob/main/ti_1_practica4_sofia_rodriguez.py